In [13]:
%load_ext autoreload
%autoreload 2

from src.llm.openai_utils import call_gpt
import os 
from src.utils.pdf_utils import read_pdf_text
    

pdf_path = "data/case/arbeidsgivers_utalelse.pdf"
pdf_path = "data/case/sykepenger_fullt_kapittel.pdf"

text = read_pdf_text(pdf_path)
print(text)



ERROR:src.utils.pdf_utils:Error reading PDF data/case/sykepenger_fullt_kapittel.pdf: no such file: 'data/case/sykepenger_fullt_kapittel.pdf'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



In [10]:
response = call_gpt("","Hi", model="gpt-4o-mini")

print(response)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hello! How can I assist you today?


In [16]:
# Now gather all the pdfs in the data/case folder and store them in a dictionary of {pdf_name: text}

pdf_files = [f for f in os.listdir("data") if f.endswith(".pdf")]
pdf_dict = {f: read_pdf_text(os.path.join("data", f)) for f in pdf_files}



# Create a function that turns these into a singles stirng on the format "### pdf_name [i]\npdf_text \n\n ### pdf_name [i+1]\npdf_text \n\n etc"

def pdf_to_string(pdf_dict):
    return "\n\n".join([f"### {pdf_name} [{i}]\n{pdf_text}" for i, (pdf_name, pdf_text) in enumerate(pdf_dict.items())])

pdf_string = pdf_to_string(pdf_dict)
#print(pdf_string)
print(len(pdf_string))

# Create a system propmt that instructs the model to be an information combiner that purely helps gather all relevant information from the pdfs to create a summary document with everything relevant for the case:

system_prompt = """
You are an information combiner that purely helps gather all relevant information from the pdfs to create a summary document with everything relevant for the case.
You perform citations with a square bracket after all the text you are citing to indicate which pdf it came from. 
"""

user_prompt = """
Gather all the information that is relevant to treat the case of the person applying for sykepenger. 
"""

user_prompt = f"""PDF Documents:/n {pdf_string} \n\n -----\n\nWhat to extract from the documents: {user_prompt}"""

response = call_gpt(system_prompt, user_prompt, model="gpt-4o-mini")

print(response)

126324


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### Summary Document for Sykepenger Application Case

**Applicant Information:**
- Name: [Person 1's full name]
- Date of Birth: [Fiktiv fødselsdato]
- Personnummer: [Fiktivt personnummer]
- Position: [Person 1's position]
- Employer: [Person 1's employer]
- Start Date of Sickness: [Startdato for sykemelding]

**Reason for Sickness:**
The applicant has reported severe vision problems, which include:
- Tåkete syn: Difficulty focusing, especially when reading or using a computer.
- Lysfølsomhet: Increased sensitivity to bright light causing pain in the eyes.
- Dobbeltsyn: Experiencing double vision, complicating precision work and safe navigation.
- Hyppig hodepine: Frequent headaches likely associated with the vision issues.

These symptoms have progressively worsened, rendering the applicant unable to perform their job duties effectively. A consultation with [Navn på optiker/lege] on [Dato for legetime] confirmed the need for sick leave to undergo necessary examinations and treatment. 

In [18]:

# Make each sources in the answer e.g. [1] or [2] etc. a markdown hyperlink to the source e.g. [[1]](https://www.google.com)
def make_hyperlinks(response):
    for i, (pdf_name, pdf_text) in enumerate(pdf_dict.items()):
        response = response.replace(f"[{i}]", f"[[{i}]]({pdf_name})")
    return response

print(make_hyperlinks(response))

### Summary Document for Sykepenger Application Case

**Applicant Information:**
- Name: [Person 1's full name]
- Date of Birth: [Fiktiv fødselsdato]
- Personnummer: [Fiktivt personnummer]
- Position: [Person 1's position]
- Employer: [Person 1's employer]
- Start Date of Sickness: [Startdato for sykemelding]

**Reason for Sickness:**
The applicant has reported severe vision problems, which include:
- Tåkete syn: Difficulty focusing, especially when reading or using a computer.
- Lysfølsomhet: Increased sensitivity to bright light causing pain in the eyes.
- Dobbeltsyn: Experiencing double vision, complicating precision work and safe navigation.
- Hyppig hodepine: Frequent headaches likely associated with the vision issues.

These symptoms have progressively worsened, rendering the applicant unable to perform their job duties effectively. A consultation with [Navn på optiker/lege] on [Dato for legetime] confirmed the need for sick leave to undergo necessary examinations and treatment. 

In [ ]:

# Make each sources in the answer e.g. [1] or [2] etc. a markdown hyperlink to the source e.g. [[1]](https://www.google.com)
def make_hyperlinks(response):
    for i, (pdf_name, pdf_text) in enumerate(pdf_dict.items()):
        response = response.replace(f"[{i}]", f"[[{i}]]({pdf_name})")
    return response

print(make_hyperlinks(response))



In [19]:
# SAve response as a hardcoded .txt in the response folder 

with open("response/response.txt", "w") as f:
    f.write(response)

